In [1]:
import pandas as pd
import json
from pathlib import Path
import numpy as np
from collections import defaultdict

In [2]:
lines = Path('/home/bj2/faro/experiments/top9_twitter_1_1600_avgproc_min_int5m_reduced_6hr_augmented/input.json').read_text().splitlines()

dataset = defaultdict(list)

for line in lines:
    obj = json.loads(line)
    for cluster_name, v in obj["counts"].items():
        dataset[cluster_name].append(v["classifier"])

In [3]:
new_dataset = {}

for cluster_name, values in dataset.items():
    tss = []
    data = []
    for i, v in enumerate(values):
        new_dist = np.random.poisson(lam=v/60, size=60)
        # print(new_dist)
        diff = v - new_dist.sum()
        # print(diff)
        if diff < 0:
            for idx in np.random.choice(60, -diff):
                new_dist[idx] -= 1
        elif diff > 0:
            for idx in np.random.choice(60, diff):
                new_dist[idx] += 1
        # print(new_dist)
        assert new_dist.sum() == v
        ts = (60 * i + np.arange(60)) * 1e3
        tss.extend(ts.astype(int))
        data.extend(new_dist.astype(float))
    df = pd.DataFrame({"timestamp": tss, "data": data})
    new_dataset[cluster_name] = df
    assert df["data"].sum() == np.sum(values)

In [4]:
for cluster_name, df in new_dataset.items():
    df.to_csv(f"./traces/{cluster_name}.csv", index=False)